In [14]:
from dash import Dash, html, dcc, callback, Output, Input, State, dash_table
import pandas as pd
import sqlite3

import plotly.express as px
import numpy as np
from datetime import datetime

### SQL Request

In [15]:
conn = sqlite3.connect('../data/database.db')
conn_memory = sqlite3.connect(':memory:')

start_date = '2024-01-01'
end_date = '2024-01-15'

query = f'SELECT \
        Node, \
        SUM(Congestion) as Congestion \
        FROM NODES_CONGESTION_DAILY \
        WHERE Date >= "{start_date}" AND Date <= "{end_date}" \
        GROUP BY Node \
        ORDER BY Congestion DESC;'
df = pd.read_sql_query(query, conn)
df.to_sql('NODES_CONGESTION_DAILY', conn_memory, if_exists='replace', index=False)
display(df)

limit = 20
query = f'SELECT DISTINCT \
            a.Node as Source, \
            b.Node as Sink, \
            b.Congestion - a.Congestion as Congestion \
        FROM NODES_CONGESTION_DAILY a \
        CROSS JOIN NODES_CONGESTION_DAILY b \
        WHERE a.Node < b.Node OR a.Node > b.Node \
        ORDER BY Congestion DESC\
        LIMIT {limit};'
df = pd.read_sql_query(query, conn_memory)
df.to_sql('NODES_PAIR_CONGESTION', conn_memory, if_exists='replace', index=False)

display(df)

,Node,Congestion
0,WAUE.BEPM.LCS3,56990.6920
1,WAUE.BEPM.LCS2,56990.6920
2,WAUE.BEPM.LCS1,56990.6920
3,WAUE.VOLT.0172,39336.5797
4,WAUE.BEPM.LCS6,38473.9268
...,...,...
1195,OKGE.GDSM.MINCO4,-9412.7880
1196,MINCOIII,-9412.7880
1197,MPS.ROCKCREEK,-11694.9991
1198,MPS.AMUE.OUTLAW,-11694.9991


,Source,Sink,Congestion
0,MPS.OSBORN,WAUE.BEPM.LCS3,70160.5705
1,MPS.OSBORN,WAUE.BEPM.LCS2,70160.5705
2,MPS.OSBORN,WAUE.BEPM.LCS1,70160.5705
3,MPS.ROCKCREEK,WAUE.BEPM.LCS3,68685.6911
4,MPS.ROCKCREEK,WAUE.BEPM.LCS2,68685.6911
5,MPS.ROCKCREEK,WAUE.BEPM.LCS1,68685.6911
6,MPS.AMUE.OUTLAW,WAUE.BEPM.LCS3,68685.6911
7,MPS.AMUE.OUTLAW,WAUE.BEPM.LCS2,68685.6911
8,MPS.AMUE.OUTLAW,WAUE.BEPM.LCS1,68685.6911
9,OKGE_MINCO2,WAUE.BEPM.LCS3,66403.4800


In [16]:
start_date = '2024-01-01'
end_date = '2025-01-01'

node_source = 'MPS.OSBORN'
node_sink = 'WAUE.BEPM.LRSE'

# node_a = 'WAUE.BEPM.LCS1'
# node_b = 'MPS.OSBORN'

query = f'SELECT * \
        FROM NODES_CONGESTION_MONTHLY\
        WHERE Node == "{node_source}" ;'
df = pd.read_sql_query(query, conn)
df.to_sql('NODES_CONGESTION_MONTHLY_BAR_SOURCE', conn_memory, if_exists='replace', index=False)
display(df)

query = f'SELECT * \
        FROM NODES_CONGESTION_MONTHLY\
        WHERE Node == "{node_sink}" ;'
df = pd.read_sql_query(query, conn)
df.to_sql('NODES_CONGESTION_MONTHLY_BAR_SINK', conn_memory, if_exists='replace', index=False)
display(df)

limit = 100
query = f'SELECT source.Date, source.Congestion - sink.Congestion as Congestion \
        FROM NODES_CONGESTION_MONTHLY_BAR_SOURCE as sink\
        FULL OUTER JOIN NODES_CONGESTION_MONTHLY_BAR_SINK as source \
        ON source.Date == sink.Date \
        ORDER BY source.Date;'
df = pd.read_sql_query(query, conn_memory)
df.to_sql('NODES_PAIR_CONGESTION_MONTHLY', conn_memory, if_exists='replace', index=False)

display(df)

,Date,Node,Congestion
0,2024-01-01 00:00:00,MPS.OSBORN,-35478.7236
1,2024-02-01 00:00:00,MPS.OSBORN,-3479.5121
2,2024-03-01 00:00:00,MPS.OSBORN,-4551.1030
3,2024-04-01 00:00:00,MPS.OSBORN,-4166.6809
4,2024-05-01 00:00:00,MPS.OSBORN,-1240.1280
5,2024-06-01 00:00:00,MPS.OSBORN,-1879.4019
6,2024-07-01 00:00:00,MPS.OSBORN,-919.6705
7,2024-08-01 00:00:00,MPS.OSBORN,-3420.1996
8,2024-09-01 00:00:00,MPS.OSBORN,-4780.1531
9,2024-10-01 00:00:00,MPS.OSBORN,-7764.9557


,Date,Node,Congestion
0,2024-01-01 00:00:00,WAUE.BEPM.LRSE,-16214.9138
1,2024-02-01 00:00:00,WAUE.BEPM.LRSE,-6498.7280
2,2024-03-01 00:00:00,WAUE.BEPM.LRSE,-8284.2528
3,2024-04-01 00:00:00,WAUE.BEPM.LRSE,-12644.3611
4,2024-05-01 00:00:00,WAUE.BEPM.LRSE,-9326.5978
5,2024-06-01 00:00:00,WAUE.BEPM.LRSE,-9370.2308
6,2024-07-01 00:00:00,WAUE.BEPM.LRSE,-10060.6837
7,2024-08-01 00:00:00,WAUE.BEPM.LRSE,-10731.9004
8,2024-09-01 00:00:00,WAUE.BEPM.LRSE,-9483.0876
9,2024-10-01 00:00:00,WAUE.BEPM.LRSE,-16160.3418


,Date,Congestion
0,2024-01-01 00:00:00,19263.8098
1,2024-02-01 00:00:00,-3019.2159
2,2024-03-01 00:00:00,-3733.1498
3,2024-04-01 00:00:00,-8477.6802
4,2024-05-01 00:00:00,-8086.4698
5,2024-06-01 00:00:00,-7490.8289
6,2024-07-01 00:00:00,-9141.0132
7,2024-08-01 00:00:00,-7311.7008
8,2024-09-01 00:00:00,-4702.9345
9,2024-10-01 00:00:00,-8395.3861


In [17]:
start_date = '2024-01-01'
end_date = '2024-01-31'

node_source = 'MPS.OSBORN'
node_sink = 'WAUE.BEPM.LRSE'

node_source = 'AECC_CSWS'
node_sink = 'AECC_CSWS'

# node_a = 'WAUE.BEPM.LCS1'
# node_b = 'MPS.OSBORN'

query = f'SELECT * \
        FROM NODES_CONGESTION_DAILY\
        WHERE Node == "{node_source}" \
        AND Date >= "{start_date}" AND Date <= "{end_date}" ;'
df = pd.read_sql_query(query, conn)
df.to_sql('NODES_CONGESTION_DAILY_BAR_SOURCE', conn_memory, if_exists='replace', index=False)
display(df)

query = f'SELECT * \
        FROM NODES_CONGESTION_DAILY\
        WHERE Node == "{node_sink}" \
        AND Date >= "{start_date}" AND Date <= "{end_date}" ;'
df = pd.read_sql_query(query, conn)
df.to_sql('NODES_CONGESTION_DAILY_BAR_SINK', conn_memory, if_exists='replace', index=False)
display(df)

limit = 100
query = f'SELECT source.Date, source.Congestion - sink.Congestion as Congestion \
        FROM NODES_CONGESTION_DAILY_BAR_SOURCE as sink\
        FULL OUTER JOIN NODES_CONGESTION_DAILY_BAR_SINK as source \
        ON source.Date == sink.Date \
        ORDER BY source.Date;'
df = pd.read_sql_query(query, conn_memory)
df.to_sql('NODES_PAIR_CONGESTION_DAILY', conn_memory, if_exists='replace', index=False)

display(df)

,Date,Node,Congestion
0,2024-01-01 00:00:00,MPS.OSBORN,-27.9209
1,2024-01-02 00:00:00,MPS.OSBORN,-1034.1456
2,2024-01-03 00:00:00,MPS.OSBORN,-267.9284
3,2024-01-04 00:00:00,MPS.OSBORN,-504.1592
4,2024-01-05 00:00:00,MPS.OSBORN,-312.6807
5,2024-01-06 00:00:00,MPS.OSBORN,-283.9191
6,2024-01-07 00:00:00,MPS.OSBORN,-244.9204
7,2024-01-08 00:00:00,MPS.OSBORN,-740.2739
8,2024-01-09 00:00:00,MPS.OSBORN,-610.3438
9,2024-01-10 00:00:00,MPS.OSBORN,-358.8615


,Date,Node,Congestion
0,2024-01-01 00:00:00,WAUE.BEPM.LRSE,-8.9024
1,2024-01-02 00:00:00,WAUE.BEPM.LRSE,-158.5257
2,2024-01-03 00:00:00,WAUE.BEPM.LRSE,-155.5955
3,2024-01-04 00:00:00,WAUE.BEPM.LRSE,-382.8806
4,2024-01-05 00:00:00,WAUE.BEPM.LRSE,-124.3880
5,2024-01-06 00:00:00,WAUE.BEPM.LRSE,-132.5412
6,2024-01-07 00:00:00,WAUE.BEPM.LRSE,-63.7916
7,2024-01-08 00:00:00,WAUE.BEPM.LRSE,-938.1757
8,2024-01-09 00:00:00,WAUE.BEPM.LRSE,-713.3502
9,2024-01-10 00:00:00,WAUE.BEPM.LRSE,-357.7036


,Date,Congestion
0,2024-01-01 00:00:00,19.0185
1,2024-01-02 00:00:00,875.6199
2,2024-01-03 00:00:00,112.3329
3,2024-01-04 00:00:00,121.2786
4,2024-01-05 00:00:00,188.2927
5,2024-01-06 00:00:00,151.3779
6,2024-01-07 00:00:00,181.1288
7,2024-01-08 00:00:00,-197.9018
8,2024-01-09 00:00:00,-103.0064
9,2024-01-10 00:00:00,1.1579


In [18]:
query = f'SELECT DISTINCT Node \
        FROM NODES_CONGESTION_MONTHLY;'
df = pd.read_sql_query(query, conn)
display(df['Node'].tolist())

['AEC',
 'AECC_CSWS',
 'AECC_ELKINS',
 'AECC_FITZHUGH',
 'AECC_FLTCREEK',
 'AECC_FULTON',
 'AECC_HYDRO13',
 'AECC_JWTURK',
 'AECI',
 'AEPM_CSWS',
 'ALTW',
 'AMRN',
 'ANTELOPE',
 'BBA',
 'BCA',
 'BEPM_RM_NPPD',
 'BEPM_TS_NPPD',
 'BLKW',
 'BLUECANYON2',
 'BLUECANYON5',
 'BLUECANYON6',
 'BRAZ',
 'BUBA',
 'BUFFALO_DUNES_WIND',
 'CANADIAN_HILLS_1',
 'CANADIAN_HILLS_2',
 'CANADIAN_HILLS_3',
 'CANADIAN_HILLS_4',
 'CANADIAN_HILLS_5',
 'CANEY_RIVER_WINDPOWER',
 'CASS_CO_1',
 'CASS_CO_2',
 'CLEC',
 'COFFEYVILLE_7',
 'COWP',
 'COWP_GEN1',
 'COWP_GEN2',
 'CPLE',
 'CPLW',
 'CROSSROADS1',
 'CROSSROADS2',
 'CROSSROADS3',
 'CROSSROADS4',
 'CRTH_SPA',
 'CSWARSENALHILL5',
 'CSWCOMANCHE1',
 'CSWETTURK',
 'CSWFLINTCREEK1',
 'CSWJLSTALL',
 'CSWKNOXLEE5',
 'CSWLIEBERMAN3',
 'CSWLIEBERMAN4',
 'CSWMATTISON1',
 'CSWMATTISON2',
 'CSWMATTISON3',
 'CSWMATTISON4',
 'CSWMSTURK',
 'CSWNARROWS1',
 'CSWNORTHEASTERN1',
 'CSWNORTHEASTERN2',
 'CSWNORTHEASTERN3',
 'CSWPIRKEY1',
 'CSWRIVERSIDE1',
 'CSWRIVERSIDE2',
 'CSWRIV

### Dashboard elements

In [19]:
app = Dash(__name__)

# Lire les données initiales
df = pd.read_sql_query('SELECT * FROM NODES_CONGESTION_DAILY ORDER BY Congestion DESC', conn)
df['Date'] = pd.to_datetime(df['Date'])  # Assurez-vous que la colonne date est au bon format

app.layout = html.Div([
    html.H1("Dashboard pour les données de congestion des noeuds"),
    
    # Conteneur pour les contrôles
    html.Div([
        # Sélecteur de dates
        html.Div([
            html.Label("Date de début:"),
            dcc.DatePickerSingle(
                id='date-debut',
                min_date_allowed=df['Date'].min(),
                max_date_allowed=df['Date'].max(),
                initial_visible_month=df['Date'].min(),
                date=df['Date'].min(),
                style={'marginRight': '20px'}
            ),
            
            html.Label("Date de fin:"),
            dcc.DatePickerSingle(
                id='date-fin',
                min_date_allowed=df['Date'].min(),
                max_date_allowed=df['Date'].max(),
                initial_visible_month=df['Date'].max(),
                date=df['Date'].max(),
                style={'marginRight': '20px'}
            ),
            html.Label("Nombre maximum de lignes:"),
            dcc.Input(
                id='max-rows',
                type='number',
                value=10,
                min=1,
                max=len(df),
                style={'marginLeft': '10px', 'width': '100px'}
            ),
        ], style={'marginBottom': '20px'}),
        
        # Bouton pour appliquer les filtres
        html.Button('Appliquer les filtres', id='apply-button', 
                   style={'backgroundColor': '#4CAF50',
                          'color': 'white',
                          'padding': '10px 20px',
                          'border': 'none',
                          'borderRadius': '5px',
                          'cursor': 'pointer'})
    ], style={'padding': '20px', 'backgroundColor': '#f8f9fa', 'borderRadius': '5px'}),
    
    # Tableau
    dash_table.DataTable(
        id='table',
        columns=[
            {'name': col, 'id': col} for col in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '10px'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        sort_action='native',  # Permet le tri
        filter_action='native'  # Permet le filtrage
    )
])

@app.callback(
    Output('table', 'data'),
    Output('table', 'page_size'),
    [Input('apply-button', 'n_clicks')],
    [State('date-debut', 'date'),
     State('date-fin', 'date'),
     State('max-rows', 'value')]
)
def update_table(n_clicks, date_debut, date_fin, max_rows):
    # Convertir les dates en datetime
    date_debut = pd.to_datetime(date_debut)
    date_fin = pd.to_datetime(date_fin)
    
    # Filtrer les données
    filtered_df = df[
        (df['Date'] >= date_debut) &
        (df['Date'] <= date_fin)
    ].sort_values('Congestion', ascending=False)
    
    # Limiter le nombre de lignes
    filtered_df = filtered_df.head(max_rows)
    
    return filtered_df.to_dict('records'), max_rows

if __name__ == '__main__':
    app.run(debug=True, port=5050)

In [20]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import sqlite3

# Initialiser l'application
app = Dash(__name__)

conn = sqlite3.connect('../data/database.db')
# Lire les données
df = pd.read_sql_query('SELECT * FROM NODES_CONGESTION_DAILY ORDER BY Congestion DESC', conn)
df['Date'] = pd.to_datetime(df['Date'])

app.layout = html.Div([
    html.H1("Analyse des Congestions", style={'textAlign': 'center'}),
    
    # Contrôles
    html.Div([
        # Sélecteur de date
        html.Div([
            html.Label("Sélectionner la période:"),
            dcc.DatePickerRange(
                id='date-range',
                min_date_allowed=df['Date'].min(),
                max_date_allowed=df['Date'].max(),
                start_date=df['Date'].min(),
                end_date=df['Date'].max(),
                style={'marginLeft': '10px'}
            ),
        ], style={'marginBottom': '20px'}),
        
        # Sélecteur du nombre de nœuds top N
        html.Div([
            html.Label("Nombre de nœuds à afficher:"),
            dcc.Slider(
                id='top-n-slider',
                min=5,
                max=50,
                step=5,
                value=10,
                marks={i: str(i) for i in range(5, 51, 5)},
                # style={'width': '500px', 'marginLeft': '10px'}
            ),
        ]),
    ], style={'padding': '20px', 'backgroundColor': '#f8f9fa', 'borderRadius': '5px'}),
    
    # Graphique
    html.Div([
        dcc.Graph(id='congestion-bar-chart')
    ], style={'marginTop': '20px'})
])

@app.callback(
    Output('congestion-bar-chart', 'figure'),
    [Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('top-n-slider', 'value')]
)
def update_graph(start_date, end_date, top_n):
    # Filtrer les données par date
    filtered_df = df[
        (df['Date'] >= start_date) &
        (df['Date'] <= end_date)
    ]
    
    # Agréger les données par nœud
    agg_df = filtered_df.groupby('Node')['Congestion'].sum().reset_index()
    
    # Prendre les top N nœuds
    top_nodes = agg_df.nlargest(top_n, 'Congestion')
    
    # Créer le graphique
    fig = px.bar(
        top_nodes,
        x='Node',
        y='Congestion',
        title=f'Top {top_n} Nœuds par Congestion',
        labels={'Node': 'Nœud', 'Congestion': 'Congestion Totale'},
        color='Congestion',
        color_continuous_scale='Viridis'
    )
    
    # Personnaliser le layout
    fig.update_layout(
        xaxis_tickangle=-45,
        bargap=0.2,
        height=600,
        title_x=0.5,  # Centrer le titre
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGray'
        ),
        yaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGray'
        )
    )
    
    return fig

if __name__ == '__main__':
    app.run(debug=True, port=6050)